In [1]:
import sys
import os
main_path = os.path.abspath(os.path.join(os.path.dirname("./test_pipeline.ipynb"), '..'))
sys.path.insert(0, main_path)

In [69]:
from src.preprocesing import gen_dataset, gen_dataset_from_h5
from src.pipelines import PipelineBuilder
from src.feature_extraction.text import MPTextGenerator, CountVectorizer


import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
from scipy import sparse
import time
import pandas as pd

from sklearn.model_selection import cross_val_score, GridSearchCV

In [3]:
merged_labels_to_num = {
    "Single microlens": 1,
    "TDE": 2,
    "Short period VS": 3,
    "SN": 4,
    "M-dwarf": 5,
    "AGN": 6,
    "Unknown": 99
}

merged_labels = {
    6: "Single microlens",
    15: "TDE",
    16: "Short period VS",
    42: "SN",
    52: "SN",
    53: "Short period VS",
    62: "SN",
    64: "SN",
    65: "M-dwarf",
    67: "SN",
    88: "AGN",
    90: "SN",
    92: "Short period VS",
    95: "SN",
    99: "Unknown"
}

res, labels, metadata = gen_dataset_from_h5("plasticc_balanced_combined_classes_small_ddf")
bands = ["lsstg", "lssti", "lsstr", "lsstu", "lssty", "lsstz"]
spatial_comp = int(np.mean([len(ts.observations["flux"]) * 2 for ts in res]))
time_durations = np.array([ts.observations["time"].to_numpy()[-1] - ts.observations["time"].to_numpy()[0] for ts in res])
mean_time = np.mean(time_durations)
std_time = np.std(time_durations)
print(mean_time, std_time, spatial_comp)

k = spatial_comp // len(res)
print("target k:", k)
labels_merged = np.array([merged_labels_to_num[merged_labels[x]] for x in labels])
print("classes:", np.unique(labels_merged))

Object: 100%|██████████| 600/600 [00:00<00:00, 239059.79it/s]

601.5589505662784 299.43984508617797 463
target k: 0
classes: [1 2 3 4 5 6]


# Pipeline for precomputed bag of words

In [18]:
alph_size = 4
quantity = "mean"
num_reduction=True
irr_handler="supp_interp"
index_based_paa = False
mean_bp_dist="normal"
verbose=False
_BANDS = ["lsstg", "lssti", "lsstr", "lsstu", "lssty", "lsstz"]
n_jobs = 8
limit = int(len(res) * 0.05)
limit
# limit = 0

30

In [19]:
if __name__ == "__main__":
    wls = [2, 3, 4, 5, 6]
    wins = (mean_time + std_time) * 10 ** np.linspace(-1.95, 0, 40)
    values = []
    out_wins = []
    out_wls = []
    ini = time.time()
    for wl in wls:
        for win in wins:
            threshold = max(1, int(round(wl / 2)))
            tol = wl
            gen = MPTextGenerator(bands=_BANDS, n_jobs=n_jobs, alph_size=4,
                                 quantity=quantity, num_reduction=num_reduction,
                                 irr_handler=irr_handler, index_based_paa=index_based_paa,
                                 mean_bp_dist=mean_bp_dist, verbose=verbose, win=win, word_length=wl)
            vec = CountVectorizer(alph_size=alph_size, word_length=wl, empty_handler=irr_handler, bands=_BANDS)
            corpus = np.array(gen.transform(res))
            fails = 0
            for c in corpus:
                if c is None:
                    fails += 1
                    
            if fails > limit:
                print("%d>%s time series failed to be represented, dropping sequence" % (fails, limit))
                continue
            matrix = sparse.csr_matrix(vec.transform(corpus))
            values.append(matrix)
            out_wins.append(win)
            out_wls.append(wl)
#             break
#         break
    end = time.time()
print("TIME: ", end - ini)


160>30 time series failed to be represented, dropping sequence



155>30 time series failed to be represented, dropping sequence



306>30 time series failed to be represented, dropping sequence



324>30 time series failed to be represented, dropping sequence



324>30 time series failed to be represented, dropping sequence


...


600>30 time series failed to be represented, dropping sequence



600>30 time series failed to be represented, dropping sequence



596>30 time series failed to be represented, dropping sequence



579>30 time series failed to be represented, dropping sequence



595>30 time series failed to be represented, dropping sequence
TIME:  891.3609461784363


In [86]:
class_based = True
class_type = "type-2"
classes = np.unique(labels_merged)

norm_key="l2"
use_idf=True
smooth_idf=True
sublinear_tf=True

reducer_type = "lsa"
n_iter = 20

n_neighbors=1
# print(spatial_comp)
pipeline_builder = PipelineBuilder(class_based=class_based, class_type=class_type, classes=classes)

pipeline_builder.set_feature_extraction(precomputed=True, idx=-1, data=values, win_arr=out_wins, wl_arr=out_wls)
pipeline_builder.set_transformer(norm=norm_key, use_idf=use_idf, smooth_idf=smooth_idf, sublinear_tf=sublinear_tf)
pipeline_builder.set_reducer(spatial_complexity=spatial_comp, reducer_type=reducer_type, n_iter=n_iter)
# print(pipeline_builder.pipe)
pipeline_builder.set_normalizer()
pipeline_builder.set_classifier(n_neighbors=n_neighbors)
pipeline = pipeline_builder.build()

In [49]:
out_wins[53], out_wls[53]

(80.30160218826266, 5)

In [28]:
spatial_comp

463

In [48]:
spatial_comp - np.logspace(np.log10(1), np.log10(spatial_comp-1), 40, dtype=int)

array([462, 462, 462, 462, 462, 461, 461, 460, 460, 459, 459, 458, 457,
       456, 454, 453, 451, 449, 447, 444, 440, 436, 432, 426, 420, 412,
       404, 394, 382, 368, 351, 332, 310, 284, 253, 217, 175, 126,  69,
         1])

In [99]:
np.linspace(10, spatial_comp-1, 10, dtype=int)

array([ 10,  60, 110, 160, 210, 261, 311, 361, 411, 462])

In [53]:
parameters = {
    "ext__idx": np.array([53]),
    "red__sc": np.arange(1, 1 + (spatial_comp//10)) * 10
}

X = np.arange(len(res))
if __name__ == "__main__":
    grid_search = GridSearchCV(pipeline, parameters, n_jobs=8, verbose=1, cv=10)
    t0 = time.time()
    grid_search.fit(X, labels_merged)
    print("DONE IN %0.3fs" % (time.time() - t0))
    print("Best parameters set found on development set:")
    print()
    print(grid_search.best_params_)
    print()
    print("Grid scores on development set:")
    print()
    means = grid_search.cv_results_["mean_test_score"]
    stds = grid_search.cv_results_["std_test_score"]
    for mean, std, params in zip(means, stds, grid_search.cv_results_["params"]):
        print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params))
    print()
    
#     print("Detailed classification report:")
#     print()
#     print("The model is trained on the full development set.")
#     print("The scores are computed on the full evaluation set.")
#     print()
#     y_true, y_pred = y_test, grid_search.predict(X_test)

Fitting 10 folds for each of 46 candidates, totalling 460 fits


[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:   11.7s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:  1.5min
[Parallel(n_jobs=8)]: Done 434 tasks      | elapsed:  7.3min
[Parallel(n_jobs=8)]: Done 460 out of 460 | elapsed:  8.2min finished


gettign the count words for win idx: 80.301602.3 and wl idx: 5
update n components in fit_transform to: 259
DONE IN 494.012s
Best parameters set found on development set:

{'ext__idx': 53, 'red__sc': 260}

Grid scores on development set:

0.498 (+/-0.123) for {'ext__idx': 53, 'red__sc': 10}
0.520 (+/-0.094) for {'ext__idx': 53, 'red__sc': 20}
0.527 (+/-0.095) for {'ext__idx': 53, 'red__sc': 30}
0.543 (+/-0.117) for {'ext__idx': 53, 'red__sc': 40}
0.542 (+/-0.099) for {'ext__idx': 53, 'red__sc': 50}
0.547 (+/-0.095) for {'ext__idx': 53, 'red__sc': 60}
0.542 (+/-0.095) for {'ext__idx': 53, 'red__sc': 70}
0.543 (+/-0.098) for {'ext__idx': 53, 'red__sc': 80}
0.533 (+/-0.106) for {'ext__idx': 53, 'red__sc': 90}
0.535 (+/-0.115) for {'ext__idx': 53, 'red__sc': 100}
0.532 (+/-0.122) for {'ext__idx': 53, 'red__sc': 110}
0.532 (+/-0.124) for {'ext__idx': 53, 'red__sc': 120}
0.547 (+/-0.111) for {'ext__idx': 53, 'red__sc': 130}
0.548 (+/-0.104) for {'ext__idx': 53, 'red__sc': 140}
0.540 (+/-0.10

In [83]:
grid_search.cv_results_.keys()

dict_keys(['mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time', 'param_ext__idx', 'param_red__sc', 'params', 'split0_test_score', 'split1_test_score', 'split2_test_score', 'split3_test_score', 'split4_test_score', 'split5_test_score', 'split6_test_score', 'split7_test_score', 'split8_test_score', 'split9_test_score', 'mean_test_score', 'std_test_score', 'rank_test_score'])

In [106]:
n = len(grid_search.cv_results_["params"])
c = class_type if class_based else None
# c = None
scheme = "l" if sublinear_tf else "n"
scheme += "t" if use_idf else "n"
scheme += "c" if norm_key == "l2" else "n"
scheme += "." + scheme

data_dict = {
    "class_feature": np.full(n, c),
    "scheme": np.full(n, scheme),
#     "smooth_idf": np.full(n, smooth_idf),
    "reducer_type": np.full(n, reducer_type),
#     "n_neighbors": np.full(n, n_neighbors),
}
for k in grid_search.param_grid.keys():
    data_dict["param_" + k] = grid_search.cv_results_["param_" + k]
    
data_dict["window"] = np.array([out_wins[k] for k in data_dict["param_ext__idx"]])
data_dict["word_length"] = np.array([out_wls[k] for k in data_dict["param_ext__idx"]])
data_dict["mean_test_score"] = grid_search.cv_results_["mean_test_score"]
data_dict["std_test_score"] = grid_search.cv_results_["std_test_score"]
data_dict["rank_test_score"] = grid_search.cv_results_["rank_test_score"]

df = pd.DataFrame(data_dict)
df

,class_feature,scheme,reducer_type,param_ext__idx,param_red__sc,window,word_length,mean_test_score,std_test_score,rank_test_score
0,type-2,ltc.ltc,lsa,53,10,80.301602,5,0.498333,0.061667,46
1,type-2,ltc.ltc,lsa,53,20,80.301602,5,0.520000,0.047022,45
2,type-2,ltc.ltc,lsa,53,30,80.301602,5,0.526667,0.047258,44
3,type-2,ltc.ltc,lsa,53,40,80.301602,5,0.543333,0.058310,33
4,type-2,ltc.ltc,lsa,53,50,80.301602,5,0.541667,0.049582,35
5,type-2,ltc.ltc,lsa,53,60,80.301602,5,0.546667,0.047610,29
6,type-2,ltc.ltc,lsa,53,70,80.301602,5,0.541667,0.047288,35
7,type-2,ltc.ltc,lsa,53,80,80.301602,5,0.543333,0.048990,33
8,type-2,ltc.ltc,lsa,53,90,80.301602,5,0.533333,0.053229,41
9,type-2,ltc.ltc,lsa,53,100,80.301602,5,0.535000,0.057470,40


## simple cross validation

In [10]:
pipeline_builder = PipelineBuilder(class_based=class_based, class_type=class_type, classes=classes)

pipeline_builder.set_feature_extraction(precomputed=True, idx=2, data=values, win_arr=out_wins, wl_arr=out_wls)
pipeline_builder.set_transformer(norm=norm, use_idf=use_idf, smooth_idf=smooth_idf, sublinear_tf=sublinear_tf)
pipeline_builder.set_reducer(spatial_complexity=spatial_comp, reducer_type=reducer_type, n_iter=n_iter)
# print(pipeline_builder.pipe)
pipeline_builder.set_normalizer()
pipeline_builder.set_classifier(n_neighbors=n_neighbors)
pipeline = pipeline_builder.build()

scores = cross_val_score(pipeline, X, labels_merged, cv=10, scoring="balanced_accuracy")
scores

463
gettign the count words for win idx: 122.477847.3 and wl idx: 2
gettign the count words for win idx: 122.477847.3 and wl idx: 2
gettign the count words for win idx: 122.477847.3 and wl idx: 2
gettign the count words for win idx: 122.477847.3 and wl idx: 2
gettign the count words for win idx: 122.477847.3 and wl idx: 2
gettign the count words for win idx: 122.477847.3 and wl idx: 2
gettign the count words for win idx: 122.477847.3 and wl idx: 2
gettign the count words for win idx: 122.477847.3 and wl idx: 2
gettign the count words for win idx: 122.477847.3 and wl idx: 2
gettign the count words for win idx: 122.477847.3 and wl idx: 2


array([0.25      , 0.2       , 0.35      , 0.4       , 0.31666667,
       0.35      , 0.3       , 0.35      , 0.31666667, 0.3       ])

# Pipeline for non-precomputed bag of words

In [ ]:
class_based = False
class_type = "type-1"
classes = np.unique(labels_merged)
precomputed=True

alph_size = 4
quantity = "mean"
num_reduction=True
irr_handler="supp_interp"
index_based_paa = False
mean_bp_dist="normal"
verbose=False
_BANDS = ["lsstg", "lssti", "lsstr", "lsstu", "lssty", "lsstz"]
n_jobs = 8
win = 100
wl = 3
threshold = 2
tol = 3


norm="l2"
use_idf=True
smooth_idf=True
sublinear_tf=True

reducer_type = "anova"
n_iter = 20

n_neighbors=1
print(spatial_comp)
pipeline_builder = PipelineBuilder(class_based=class_based, class_type=class_type, classes=classes)


# pipeline_builder.set_feature_extraction(precomputed=True, idx=-1, data=values, win_arr=out_wins, wl_arr=out_wls)

pipeline_builder.set_feature_extraction(precomputed=False, bands=_BANDS, n_jobs=n_jobs, alph_size=4,
                                        quantity=quantity, num_reduction=num_reduction,
                                        irr_handler=irr_handler, index_based_paa=index_based_paa,
                                        mean_bp_dist=mean_bp_dist, verbose=verbose, win=win, word_length=wl)


pipeline_builder.set_transformer(norm=norm, use_idf=use_idf, smooth_idf=smooth_idf, sublinear_tf=sublinear_tf)
pipeline_builder.set_reducer(spatial_comp, reducer_type=reducer_type, n_iter=n_iter)
pipeline_builder.set_normalizer()
pipeline_builder.set_classifier(n_neighbors=n_neighbors)
pipeline = pipeline_builder.build()